In [1]:
import numpy as np
import h5py
from scipy.io import loadmat
from functools import reduce

In [2]:
def Patch(data,height_index,width_index,PATCH_SIZE):   # PATCH_SIZE为一个patch（边长-1）的一半    data维度(H,W,C)
    height_slice = slice(height_index-PATCH_SIZE, height_index+PATCH_SIZE)
    width_slice = slice(width_index-PATCH_SIZE, width_index+PATCH_SIZE)
    # 由height_index和width_index定位patch中心像素
    patch = data[height_slice, width_slice,:]
    patch = patch.reshape(-1,patch.shape[0]*patch.shape[1]*patch.shape[2])
    # print(patch.shape)
    return patch

In [3]:
img = loadmat('../../dataset/3_Kennedy_Space_Center/KSC.mat')['KSC']
gt = loadmat('../../dataset/3_Kennedy_Space_Center/KSC_gt.mat')['KSC_gt']
print(img.shape)  # (512, 614, 176)

(512, 614, 176)


In [4]:
# 统计每类样本所含个数
dict_k = {}
for i in range(gt.shape[0]):
    for j in range(gt.shape[1]):
        if gt[i][j] in range(0,gt.max()+1):
            if gt[i][j] not in dict_k:
                dict_k[gt[i][j]]=0
            dict_k[gt[i][j]] +=1

print(dict_k) 
# {0: 309157, 8: 431, 12: 503, 13: 927, 11: 419, 5: 161, 1: 761, 4: 252, 6: 229, 2: 243, 3: 256, 10: 404, 7: 105, 9: 520}
print(reduce(lambda x,y:x+y,dict_k.values())) #314368

# 5211 = 314368 - 309157

{0: 309157, 8: 431, 12: 503, 13: 927, 11: 419, 5: 161, 1: 761, 4: 252, 6: 229, 2: 243, 3: 256, 10: 404, 7: 105, 9: 520}
314368


In [5]:
'''
波段处理
'''
bands_select = [1, 66, 67, 68, 72, 74, 75, 76, 77, 81, 82, 83, 84, 85, 86, 87, 91, 93, 94, 95, 96, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176]
for i in range(len(bands_select)):
    bands_select[i] = bands_select[i] - 1 
# print(bands_select)

'''
波段选择
'''
img = img[:, :, bands_select]
print(img.shape)

'''
图像归一化
'''
img = ( img * 1.0 - img.min() ) / ( img.max() - img.min() )
type(img)

(512, 614, 100)


numpy.ndarray

In [6]:
[m, n, b] = img.shape
label_num = gt.max()
print(label_num)
PATCH_SIZE = 14

13


In [7]:
# padding the hyperspectral images
img_temp = np.zeros((m + 2 * PATCH_SIZE, n + 2 * PATCH_SIZE, b), dtype=np.float32)
img_temp[PATCH_SIZE:(m + PATCH_SIZE), PATCH_SIZE:(n + PATCH_SIZE), :] = img[:, :, :]

for i in range(PATCH_SIZE):
    img_temp[i, :, :] = img_temp[2 * PATCH_SIZE - i, :, :]
    img_temp[m + PATCH_SIZE + i, :, :] = img_temp[m + PATCH_SIZE - i - 2, :, :]

for i in range(PATCH_SIZE):
    img_temp[:, i, :] = img_temp[:, 2 * PATCH_SIZE - i, :]
    img_temp[:, n + PATCH_SIZE + i, :] = img_temp[:, n + PATCH_SIZE  - i - 2, :]

img = img_temp

gt_temp = np.zeros((m + 2 * PATCH_SIZE, n + 2 * PATCH_SIZE), dtype=np.int8)
gt_temp[PATCH_SIZE:(m + PATCH_SIZE), PATCH_SIZE:(n + PATCH_SIZE)] = gt[:, :]
gt = gt_temp

img.shape, gt.shape

((540, 642, 100), (540, 642))

In [8]:
data = []
label = []
[m, n, b] = img.shape
count = 0 #统计有多少个类别不为0的pixel

In [9]:
for i in range(PATCH_SIZE, m - PATCH_SIZE):
    for j in range(PATCH_SIZE, n - PATCH_SIZE):
        if gt[i, j] == 0:
            continue
        else:
            count += 1
            temp_data = Patch(img, i, j, PATCH_SIZE)
            # temp_label = np.zeros((1, label_num), dtype=np.int8)  # temp_label为一行九列[0,1,2,....,7,8]表示类别
            temp_label = gt[i, j] - 1

            data.append(temp_data)  # 每一行表示一个patch
            label.append(temp_label)
print(count)

5211


In [10]:
data = np.array(data)
print(data.shape)
''' (5211, 1, 78400) '''

data = np.squeeze(data)
print("squeeze : ", data.shape)
''' squeeze :  (5211, 78400) '''


label = np.array(label)
print(label.shape)
''' (5211,) '''

label = np.squeeze(label)
print("squeeze : ", label.shape)
''' squeeze :  (5211,) '''

(5211, 1, 78400)
squeeze :  (5211, 78400)
(5211,)
squeeze :  (5211,)


' squeeze :  (5211,) '

In [12]:
print(type(label))
ksc_dict = {}
ksc_dict = ksc_dict.fromkeys([i for i in range(label_num)], 0)
print(ksc_dict)

for i in label:
    if i in ksc_dict:
        ksc_dict[i] += 1
print(ksc_dict)
ksc_label_sum = sum(ksc_dict.values())
print(ksc_label_sum)

<class 'numpy.ndarray'>
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0}
{0: 761, 1: 243, 2: 256, 3: 252, 4: 161, 5: 229, 6: 105, 7: 431, 8: 520, 9: 404, 10: 419, 11: 503, 12: 927}
5211


In [11]:
indices = np.arange(data.shape[0]) 
# (5211,)
shuffled_indices = np.random.permutation(indices)
# (5211,)
images = data[shuffled_indices]
# (5211, 78400)
labels = label[shuffled_indices]
# (5211,)

In [12]:
indices.shape, shuffled_indices.shape, images.shape, labels.shape

((5211,), (5211,), (5211, 78400), (5211,))

In [13]:
t_labeled = []

dict_sample = \
{'0': 309157, '8': 431, '12': 503, '13': 927, '11': 419, '5': 161, '1': 761, '4': 252, '6': 229, '2': 243, '3': 256, '10': 404, '7': 105, '9': 520}

for c in range(label_num): 
    if dict_sample[str(c+1)]<200:   # ''' 标记样本数小于200的不要'''
       pass
    else:
        i = indices[labels == c][:200]
        t_labeled += list(i)

In [14]:
t_images = images[t_labeled]
print('t_images', t_images.shape)
t_labels = labels[t_labeled]
print('t_labels', t_labels.shape)

'''
t_images (2066, 78400)
t_labels (2066,)
'''

t_images (2200, 78400)
t_labels (2200,)


'\nt_images (2066, 78400)\nt_labels (2066,)\n'

In [15]:
f = h5py.File(r'../../h5dataset_ica_bandselect/KSC-' + str(PATCH_SIZE * 2) + '-' + str(PATCH_SIZE * 2) + '-100.h5', 'w')  # 每类200个
f['data'] = images
f['label'] = labels
f.close()

f = h5py.File(r'../../h5dataset_ica_bandselect_200/KSC-' + str(PATCH_SIZE * 2) + '-' + str(PATCH_SIZE * 2) + '-100.h5', 'w')  # 每类200个
f['data'] = t_images
f['label'] = t_labels
f.close()

print("success")

success


In [16]:
#HDF5的读取
f = h5py.File('../../h5dataset_ica_bandselect/KSC-28-28-100.h5','r')   #打开h5文件  # 可以查看所有的主键  
# print(type(f))

print([key for key in f.keys()])

# print('first, we get values of x:', f['data'][:])
# print('then, we get values of y:', f['label'][:])
print(f['data'][:].shape)
print(f['label'][:].shape)

['data', 'label']
(5211, 78400)
(5211,)


In [17]:
#HDF5的读取
f = h5py.File('../../h5dataset_ica_bandselect_200/KSC-28-28-100.h5','r')   #打开h5文件  # 可以查看所有的主键  
# print(type(f))

print([key for key in f.keys()])

# print('first, we get values of x:', f['data'][:])
# print('then, we get values of y:', f['label'][:])
print(f['data'][:].shape)
print(f['label'][:].shape)

['data', 'label']
(2200, 78400)
(2200,)


In [18]:
#HDF5的读取
f = h5py.File('../../h5dataset/KSC-28-28-100.h5','r')   #打开h5文件  # 可以查看所有的主键  
# print(type(f))

print([key for key in f.keys()])

# print('first, we get values of x:', f['data'][:])
# print('then, we get values of y:', f['label'][:])
print(f['data'][:].shape)
print(f['label'][:].shape)


['data', 'label']
(2200, 78400)
(2200,)
